In [4]:
import pandas as pd
import numpy as np

Zamieniam zmienną 'rx' na zmienną kategoryczną, 'Obs' = 0, 'Lev'=1, 'Lev+FU'=2

Podział zbioru na treatment, target i feature

In [2]:
def prepare_data_to_model(path='../data/survival.csv'):
    data = pd.read_csv(path, index_col = 0).drop(['id','study','etype'], axis = 1)
    data['treatment'] = np.where(data['rx']=='Obs', 0, np.where(data['rx']=='Lev',1,2))
    data.drop('rx', axis=1, inplace = True)
    data = data.fillna(-1)
    return data.drop(['treatment','time'], axis=1), data['treatment'], data['time']

In [5]:
X, treatment, y =prepare_data_to_model()

In [6]:
X_nodup = X.drop_duplicates()#.reset_index(drop=True)
X_nodup

,sex,age,obstruct,perfor,adhere,nodes,status,differ,extent,surg,node4
1,1,43,0,0,0,5.0,1,2.0,3,0,1
3,1,63,0,0,0,1.0,0,2.0,3,0,0
5,0,71,0,0,1,7.0,1,2.0,2,0,1
7,0,66,1,0,0,6.0,1,2.0,3,1,1
9,1,69,0,0,0,22.0,1,2.0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1849,1,71,0,0,1,4.0,0,2.0,3,0,0
1851,0,72,0,0,0,1.0,0,2.0,3,0,0
1853,1,76,0,0,1,1.0,1,3.0,3,0,0
1855,0,48,1,0,0,4.0,0,2.0,3,1,1


Load model

In [7]:
import pickle

with open('pickles/opt_XGBTRegressor.pickle', 'rb') as file:
    xg = pickle.load(file)

The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.


In [8]:
te, lb, ub = xg.estimate_ate(X, treatment, y)
pred = xg.predict(X)

In [9]:
print('Average Treatment 1 Effect (XGBoost): {:.2f} ({:.2f}, {:.2f})'.format(te[0], lb[0], ub[0]))
print('Average Treatment 2 Effect (XGBoost): {:.2f} ({:.2f}, {:.2f})'.format(te[1], lb[1], ub[1]))

Average Treatment 1 Effect (XGBoost): 70.49 (25.09, 115.88)
Average Treatment 2 Effect (XGBoost): 70.89 (27.85, 113.93)


### CaterisParibus

In [12]:
from ceteris_paribus.explainer import explain
from ceteris_paribus.profiles import individual_variable_profile
from ceteris_paribus.select_data import select_neighbours
from ceteris_paribus.plots.plots import plot_notebook, plot

In [11]:
expCP = explain(model = xg, data = X, y =y, predict_function = (lambda x: xg.predict(x)[:,0]))

 You can add label using method set_label


In [13]:
for idx in X_nodup.index:
    cp_xgb = individual_variable_profile(expCP, X.loc[idx], y.loc[idx])
    plot(cp_xgb, selected_variables=['age',"nodes",'differ','surg'],height=1000, width=1000,
             plot_title=f'Tratment 1 idx ={idx}', yaxis_title='uplift',
             print_observations=False)

In [14]:
neighbours = select_neighbours(X_nodup, X.loc[1], 
    selected_variables=['nodes', 'age', 'status', 'sex'],n=100)
cp_xgb_ns = individual_variable_profile(expCP, neighbours)

# cp_xgb = individual_variable_profile(expCP, X.loc[1], y.loc[1])
# cp_xgb2 = individual_variable_profile(expCP, X.loc[3], y.loc[3])

Zmienna `surg` tak róznicuje

In [15]:
plot_notebook(cp_xgb_ns, selected_variables=["nodes"],height=1000, width=1000,
         plot_title='Tratment 1', yaxis_title='uplift',
         print_observations=True)

In [16]:
neighbours = select_neighbours(X_nodup, X.loc[9], 
    selected_variables=['nodes', 'age', 'status', 'sex'],n=15)
cp_xgb_ns = individual_variable_profile(expCP, neighbours)

In [17]:
plot_notebook(cp_xgb_ns,selected_variables=["nodes",'age'],height=600, width=1000,
         plot_title='Tratment 1', yaxis_title='uplift',
         print_observations=False,size_pdps=6, alpha_pdps=1, size=2)